# Text Mining Project Work (Group 5)

**Text Classification and Sentiment Analysis**

_Prof. Gianluca Moro, Dott. Ing. Nicola Piscaglia – DISI, University of Bologna_

**Bologna Business School** - Alma Mater Studiorum Università di Bologna

## Instructions
- The provided exercises must be executed by the students of Group 5
- At the end, the file must contain all the required results (as code cell outputs) along with all the commands necessary to reproduce them; 
- The function of every command or group of related commands
must be documented clearly and concisely. 
- The submission deadline is the 1st July 2022.
- When finished, one team member will send the notebook file (having .ipynb extension) via mail (using your BBS email account) to the teacher (nicola.piscaglia@bbs.unibo.it) indicating “[BBS Teamwork] Your last names” as subject, also keeping an own copy of the file for safety.
- You are allowed to consult the teaching material and to search the Web for quick reference. 
- If still in doubt about anything, ask the teacher
- It is severely NOT allowed to communicate with other teams. Ask the teacher for any clarification about the exercises.
- Each correctly developed point counts 2/30

## Setup

The following cell contains some necessary imports

In [5]:
import numpy as np
import pandas as pd
import gzip
import json
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
from urllib.request import urlretrieve
from statsmodels.stats.contingency_tables import mcnemar

Run the following to download the necessary files

In [6]:
def download(file, url):
    if not os.path.exists(file):
        urlretrieve(url, file)

In [7]:
download("Gift_Cards.json.gz", "https://www.dropbox.com/s/c87cjds263jt3sb/Gift_Cards.json.gz?dl=1")

In [8]:
download("Magazine_Subscriptions.json.gz", "https://www.dropbox.com/s/g6om8q8c8pvirw8/Magazine_Subscriptions.json.gz?dl=1")

In [9]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/gioel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Exercises

1) We provide in the `Gift_Cards.json.gz` file a dataset composed by several reviews posted on Amazon.com about Gift cards products. 
Each review is labeled with a score between 1 and 5 stars (represented by the ```overall``` feature).

The text of each review is represented by the ```reviewText``` feature which is going to be our input data along with the ```overall``` one.

Load 100000 random reviews putting it in a new Pandas dataframe.

In [29]:
df = pd.read_json('Gift_cards.json.gz',lines = True,compression= 'gzip')
df = df.sample(n = 100000)
df = df.reset_index()


2) Print the dataset rows number and visualize the first 5 rows.

In [30]:
df.head()

,index,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,25890,5,NaN,True,"01 22, 2016",AT2BNMCP26V6E,B004Q7CK9M,{'Gift Amount:': ' 15'},Samantha Barbise,Great!,Five Stars,1453420800,NaN
1,103467,5,NaN,True,"02 25, 2017",A12IYOMV4YWM5K,B00JDQJZWG,{'Gift Amount:': ' 50'},Princess,Nice presentation. A surprising way to give t...,Nice presentation. A surprising way to give th...,1487980800,NaN
2,21900,5,NaN,True,"02 21, 2018",A2LSIAFIOJVQ2G,B004LLIKVU,{'Gift Amount:': ' 50'},Jayson Nash,Perfect way to send a thank you to a customer!,Five Stars,1519171200,NaN
3,26694,5,NaN,True,"11 25, 2015",A21C225U9EAAPE,B004Q7CK9M,{'Gift Amount:': ' 20'},katzrule,"Love the card, will be used as Christmas gift.",Five Stars,1448409600,NaN
4,4199,5,NaN,True,"03 16, 2012",AHXBE7Y9TMMSM,B004KNWX6C,NaN,Big Al,The printable gift card was so easy to carry o...,Printable gift card,1331856000,NaN


3) Undersample the data by `overall` feature in order to obtain a class-balanced dataset.



In [31]:
rus = RandomUnderSampler(random_state=42)
df,_ = rus.fit_resample(df, df["overall"])

4) Cast the `reviewText` column to unicode string



In [33]:
df["reviewText"] = df["reviewText"].values.astype('U')

**5)** Select from data only the features named ```reviewText``` and ```overall``` putting them in a dataframe





In [34]:
df = df[['reviewText', 'overall']]

**6)** Verify the distribution of the number of stars

In [35]:
df["overall"].value_counts()

1    1104
2    1104
3    1104
4    1104
5    1104
Name: overall, dtype: int64

**7)** Remove from the dataframe the reviews rated with 3 stars.

In [37]:
df = df[df['overall']!= 3]

In [38]:
df['overall'].unique()

array([1, 2, 4, 5])

**8)** Add a `label` column to the DataFrame whose value is `"pos"` for reviews with 4 or 5 stars and `"neg"` for reviews with 1 or 2 stars.

In [39]:
def mapToLabel(value):
  if value >= 4:
    return "pos"
  elif value <= 2:
    return "neg"

df["label"] = df["overall"].apply(mapToLabel)

df.head() 

,reviewText,overall,label
0,Could not redeem.,1,neg
1,Do not buy this product!!!\nIf an error occurs...,1,neg
2,I had purchased this gift card as a gift to a ...,1,neg
3,Card was sent without being loaded with the am...,1,neg
4,WATCH OUT !!! DID NOT WORK - had to pay with m...,1,neg


**9)** Split the dataset randomly into a training set with 80% of data and a test set with the remaining 20%, stratifying the split by the `label` variable

In [40]:
y = df["label"]
X = df["reviewText"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("training set shape: " + str(X_train.shape))
print("Test set shape: " + str(X_test.shape))

training set shape: (3532,)
Test set shape: (884,)


**10)** Create a tf.idf vector space model from training reviews excluding words appearing in less than 7 documents and using only unigrams. Then, extract the document-term matrix for them.

In [41]:
vect = TfidfVectorizer(min_df=7, ngram_range=(1,1))
train_dtm = vect.fit_transform(X_train)

**11)** Train a Support Vector Machine of your choice on the training reviews with a regularization parameter equals to 5, using the representation created above

**12)** Verify the accuracy of the classifier on the test set and try to maximize it tuning the Support Vector Machine kernel and regularization factor.

**13)** Train a Deep Learning model (excluding transformer-based models like BERT) using the document-term representation built in point 10. The usage of recurrent layers is up to you.

14) Evaluate the model calculating the accuracy on test data. Try to maximize the model accuracy by tuning the neural network. 

15) Evaluate the DL trained model on 50000 random reviews from the dataset in `Magazine_Subscriptions.json.gz` file.

Hint: you have to repeat the preprocessing steps done in the previous steps for the Gift cards reviews.

16) Extra: train/fine-tune a transformer-based model (e.g. BERT) on Gift Cards training reviews and evaluate it on the Gift Cards test reviews.